# Text Mining Data Prep

In this notebook, we will prepare the data for text mining. 

let's start with a corpus

In [3]:
corpus = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "begin begun beginning begins",
    "is was were being",
    "123 the world is large 32.34"
]

## Create a term by document matrix

TfidfVectorizer and CountVectorizer both are methods for converting text data into vectors as model can process only numerical data.

### Using CountVectorizer

In CountVectorizer we only count the number of times a word appears in the document which results in biasing in favour of most frequent words. this ends up in ignoring rare words which could have helped is in processing our data more efficiently.

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd

# CountVectorizer will covert to lowercase, remove punctuation, and remove stop words - to 
# remove other things, such as numbers, use the token_pattern parameter
vectorizer = CountVectorizer(stop_words='english', lowercase=True, token_pattern="[^\W\d_]+") # [^\W\d_]+ not Word, not digit, not underscore -- see: https://regexr.com/
X = vectorizer.fit_transform(corpus)
df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df

,begin,beginning,begins,begun,document,large,second,world
0,0,0,0,0,1,0,0,0
1,0,0,0,0,2,0,1,0
2,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0
4,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,1


### Using TfidVectorizer

To overcome this problem (over emphasis on high frequency), we use TfidfVectorizer .

In TfidfVectorizer we consider overall document weightage of a word. It helps us in dealing with most frequent words. Using it we can penalize them. TfidfVectorizer weights the word counts by a measure of how often they appear in the documents.

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Like CountVectorizer, TfidfVectorizer will covert to lowercase, remove punctuation, and remove 
# stop words - to remove other things, such as numbers, use the token_pattern parameter
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, token_pattern="[^\W\d_]+")

X = vectorizer.fit_transform(corpus)

df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df

,begin,beginning,begins,begun,document,large,second,world
0,0.0,0.0,0.0,0.0,1.0000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.8538,0.000000,0.520601,0.000000
2,0.0,0.0,0.0,0.0,0.0000,0.000000,0.000000,0.000000
3,0.5,0.5,0.5,0.5,0.0000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0000,0.000000,0.000000,0.000000
5,0.0,0.0,0.0,0.0,0.0000,0.707107,0.000000,0.707107


### Word Stemming

Notice that we might benefit from finding word stems. For example, the words "beginning", "begun", and "begins" are all related to the same concept or begin. We can use the NLTK's WordNetLemmatizer to reduce words to their stems.

In [55]:
import nltk
# nltk.download('averaged_perceptron_tagger') # you only need to run this once
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize

# Define the corpus of documents
corpus = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "begin begun beginning begins",
    "is was were being",
    "123 the world is large 32.34",
    'striped striping stripped hanging hanged begin beginning loving love loved'
]

transformed_corpus = []
wnl = WordNetLemmatizer()
for document in corpus:
    transformed_document = ""
    for word, tag in pos_tag(word_tokenize(document)):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma = word
        else:
            lemma = wnl.lemmatize(word, wntag)
        transformed_document+= lemma + " "
    transformed_corpus += [transformed_document]

transformed_corpus


['This be the first document . ',
 'This document be the second document . ',
 'And this be the third one . ',
 'begin begin begin begin ',
 'be be be be ',
 '123 the world be large 32.34 ',
 'strip strip strip hang hang begin begin love love love ']

Now, let's use the TfidfVectorizer to convert our new lematized corpus into a matrix of TF-IDF features.

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Like CountVectorizer, TfidfVectorizer will covert to lowercase, remove punctuation, and remove 
# stop words - to remove other things, such as numbers, use the token_pattern parameter
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, token_pattern="[^\W\d_]+")

X = vectorizer.fit_transform(transformed_corpus)

df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df

,begin,document,hang,large,love,second,strip,world
0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.856605,0.000000,0.000000,0.000000,0.515973,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.707107
6,0.333665,0.000000,0.401965,0.000000,0.602948,0.000000,0.602948,0.000000


## Apply SVD for dimension reduction

Let's apply SVD to reduce the dimensionality of our data. 

In [57]:
from sklearn.decomposition import TruncatedSVD

In [58]:
#If you are performing Latent Semantic Analysis, recommended number of components is 100

svd = TruncatedSVD(n_components=5, n_iter=10)

In [59]:
X_svd = svd.fit_transform(X)
X_svd

array([[ 9.63484449e-01, -3.94276486e-18, -1.29260144e-19,
        -1.84389736e-20,  2.67764291e-01],
       [ 9.63484449e-01,  8.14196111e-16, -8.49422565e-18,
        -2.57015685e-16, -2.67764291e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.03896991e-15,  8.16598276e-01,  4.27354005e-17,
        -5.77206423e-01,  1.56448904e-15],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 5.96957229e-18,  7.65287087e-17,  1.00000000e+00,
        -7.64621080e-17, -1.19993969e-17],
       [-4.67671273e-16,  8.16598276e-01,  2.14439953e-16,
         5.77206423e-01,  9.41403335e-16]])

In [60]:
X_svd.shape[1]

5

In [61]:
df = pd.DataFrame(X_svd, columns=[f"svd{num:04}" for num in range(0,X_svd.shape[1])])
df

,svd0000,svd0001,svd0002,svd0003,svd0004
0,9.634844e-01,-3.942765e-18,-1.292601e-19,-1.843897e-20,2.677643e-01
1,9.634844e-01,8.141961e-16,-8.494226e-18,-2.570157e-16,-2.677643e-01
2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,-1.038970e-15,8.165983e-01,4.273540e-17,-5.772064e-01,1.564489e-15
4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5,5.969572e-18,7.652871e-17,1.000000e+00,-7.646211e-17,-1.199940e-17
6,-4.676713e-16,8.165983e-01,2.144400e-16,5.772064e-01,9.414033e-16


### Now were are ready to use this data in a model

Our data is now ready to be used in a model. If we have these documents tagged (for instance, 'good' or 'bad'), we can use this data to train a model. If we don't have the tags, we can use this data to cluster the documents - or, go through the documents manually and tag them.